In [2]:
import cv2
import keras
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
from modules import *
from utils import *
from AudioDataGenerator import *
import os
from flipGradientTF import *
########## have to import a EEG snippet from somewhere
#img_path = '/Users/fchollet/Downloads/creative_commons_elephant.jpg'
#df = pd.read_csv("E:/sleepEDFx/pat2cha3.csv")
# EEG_sample = df.iloc[1002].values
# stage = EEG_sample[-2]
# EEG_sample = EEG_sample[:-2]
# EEG_sample = np.expand_dims(EEG_sample,axis=-1)
# EEG_sample = np.expand_dims(EEG_sample, axis=0)

from keras.models import model_from_json




# Model reconstruction from JSON file
with open('E:/SleepWell/ASSC/models/justforreveiw_2019-02-23 13-46-16.482078/model.json', 'r') as f:
    model = model_from_json(f.read(), {'Scale': Scale, 'GradientReversal': GradientReversal})
# Load weights into the new model
model.load_weights(r'E:/SleepWell/ASSC/models/justforreveiw_2019-02-23 13-46-16.482078/weights.0001-0.6613.hdf5')

In [6]:
model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3000, 1)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 3000, 64)     1088        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 3000, 64)     256         conv1d_1[0][0]                   
__________________________________________________________________________________________________
scale_1 (Scale)                 (None, 3000, 64)     128         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
activation

In [14]:
model.layers[-3].hp_lambda

1

In [44]:
print(valY.shape)
#valY = np.argmax(valY, axis=-1)
valY = np.expand_dims(valY, axis=1)
print(valY.shape)
valX = valX

def accuracy_score(self,true,predY):
    match = sum(predY == true)
    return match/len(true)

def calcMetrics(self,predY,mask=None):

    if mask is None:
        mask = np.ones(shape=valY.shape)

    if not (self.valY.shape == predY.shape):
        predY = np.expand_dims(predY,axis=-1)

    true = valY[mask]
    predY = predY[mask]
    confMat = confusion_matrix(true, predY)
    match = sum(predY == true)  # total matches

    sens = []
    spec = []
    acc = []
    for each in np.unique(true).astype(int):
         # each = int(each)
        sens.append(confMat[each, each] / sum(confMat[each, :]))
        spec.append((match - confMat[each, each]) / (
        (match - confMat[each, each] + sum(confMat[:, each]) - confMat[each, each])))
        acc.append(match/(match+ sum(confMat[:, each] + sum(confMat[each, :] -2*confMat[each, each]))))

    return sens,spec,acc

def on_epoch_end(valX, valY, epoch):

    if 1:
        
        SC_mask = self.patID <= 39
        ST_mask = self.patID > 39
        
        
        SC_patID = self.patID[SC_mask]
        ST_patID = self.patID[ST_mask]
        
        for pat in np.unique(SC_patID).astype(int):
            mask = SC_patID[:, 0] == pat
            acc = self.accuracy_score(self.valY[mask],predY[mask])
            patAcc.append(acc)
        print(np.mean(patAcc))
        
        for pat in np.unique(self.patID).astype(int):
            mask = self.patID[:, 0] == pat
            acc = self.accuracy_score(self.valY[mask],predY[mask])
            patAcc.append(acc)
        print(np.mean(patAcc))
        
        
        
        
        
        
        predY = model.predict(valX, verbose=0)
        predY = np.argmax(predY, axis=-1)

        sens,spec,acc = calcMetrics(predY)

        for sens_,spec_,acc_,class_ in zip(sens,spec,acc,set(self.valY)):
            print(sens_)
            print(spec_)
            print(acc_)
        patAcc = []
        for pat in np.unique(self.patID).astype(int):
            mask = self.patID[:, 0] == pat
            acc = self.accuracy_score(self.valY[mask],predY[mask])
            patAcc.append(acc)
        print(np.mean(patAcc))

        sens,spec,acc = self.calcMetrics(predY,self.patID <= 39)
        print(np.mean(sens))
        print(np.mean(spec))
        print(np.mean(acc))

        sens,spec,acc = self.calcMetrics(predY,self.patID > 39)
        print(np.mean(sens))
        print(np.mean(spec))
        print(np.mean(acc))

(31925, 1)
(31925, 1, 1)


In [45]:
on_epoch_end(valX, valY, 3)
#mask = np.ones(shape=valY.shape)
#print(mask)

ValueError: Error when checking input: expected input_1 to have 3 dimensions, but got array with shape (31925, 3000)

In [33]:
predY = model.predict(valX, verbose=0)
predY = np.argmax(predY, axis=-1)
predY = np.expand_dims(predY, axis=1)
patAcc = []
print("printing the shape of predY")
print(predY.shape)
            # for pat in np.unique(self.patID).astype(int):
            #     mask = self.patID == pat
            #     patAcc.append(accuracy_score(self.valY[mask], predY[mask]))
            #logs['PerPatientAccuracy'] = np.mean(patAcc)

            ##################################################################
            
confMat = confusion_matrix(valY -1, predY)
print("confMat Done")
match = sum(predY == self.valY - 1)
print("number of matches")
print(match.shape)


patsens = []
patspec = []

dummyvalY = self.valY -1

for each in np.unique(dummyvalY).astype(int):
    each = int(each)
    patsens.append(confMat[each, each] / sum(confMat[each, :]))
    patspec.append((match - confMat[each, each]) / ((match - confMat[each, each] + sum(confMat[:, each]) - confMat[each, each])))

acc = match / len(valY)

patAcc = []
for pat in np.unique(patID).astype(int):
    mask = patID[:, 0] == pat
    patAcc.append(accuracy_score(valY[mask] - 1, predY[mask]))
patAccAvg = np.mean(patAcc)

perClassSens= []
perClassSpec = []
correctedtrueY = self.valY - 1
for each_class in np.unique(correctedtrueY):
    mask = correctedtrueY == each_class
    confMat = confusion_matrix(correctedtrueY[mask], predY[mask])
    perClassSens.append(confMat[each, each] / sum(confMat[each, :]))
    perClassSpec.append((match-confMat[each, each])/ ((match - confMat[each, each] + sum(confMat[:, each]) - confMat[each, each])))




             #######################   LOGGING  #############################################
#             logs['PerPatientAccuracy'] = np.mean(patAcc)
#             logs['PerPatientSensitivity'] = np.mean(patsens)
#             logs['PerPatientSpecificity'] = np.mean(patspec)

#             logs['PerClassSensitivity'] = np.mean(perClassSens)
#             logs['PerClassSpecificity'] = np.mean(perClassSpec)

#             logs['s1Specificity'] = perClassSpec[0]
#             logs['s2Specificity'] = perClassSpec[1]
#             logs['s3Specificity'] = perClassSpec[2]
#             logs['s4Specificity'] = perClassSpec[3]
#             logs['REMSpecificity'] = perClassSpec[4]
#             logs['WakeSpecificity'] = perClassSpec[5]

#             logs['s1Sensitivity'] = perClassSens[0]
#             logs['s2Sensitivity'] = perClassSens[1]
#             logs['s3Sensitivity'] = perClassSens[2]
#             logs['s4Sensitivity'] = perClassSens[3]
#             logs['REMSensitivity'] = perClassSens[4]
#             logs['WakeSensitivity'] = perClassSens[5]

ValueError: Error when checking input: expected input_1 to have 3 dimensions, but got array with shape (31925, 3000)

In [30]:
dummyY= valY
dummyX= valX

In [51]:
valY=dummyY
valX= dummyX

In [52]:
print(valY.shape)
#valY = np.argmax(valY, axis=-1)
print(valY.shape)
valY = np.expand_dims(valY, axis=1)
print(valY.shape)
mask = np.ones(shape=valY.shape).astype(int)
print(mask.shape)
print(mask)

(31925,)
(31925,)
(31925, 1)
(31925, 1)
[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [ ]:
predY.shape

In [4]:
#model.summary()
# from collections import Counter
# Counter(valY) + Counter(trainY)
from imblearn.keras import balanced_batch_generator
from imblearn.under_sampling import NearMiss
print(trainX.shape)
print(trainY.shape)

(73985, 3000)
(73985,)


In [ ]:
 training_generator, steps_per_epoch = balanced_batch_generator( trainX, 
                                                                trainY, sampler = NearMiss(), 
                                                                batch_size = 64, random_state = 42)

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from keras import backend as K

def patientSplitter(randomIDfile,df2,split_portion):
    import pandas as pd

    df1 = pd.read_csv(randomIDfile,header=None)
    split_portion_numer=int(split_portion*61)

    train_pat_list = [int(each) for each in df1.iloc[:split_portion_numer].values]
    test_pat_list = [int(each) for each in df1.iloc[split_portion_numer:].values]
    print(test_pat_list)
    df3 = []
    df4 = []
    for pat_ID in train_pat_list:
        df3.append(df2[df2.patID == pat_ID].values)
        print(pat_ID)
    for pat_ID in test_pat_list:
        df4.append(df2[df2.patID == pat_ID].values)
        print(pat_ID)
    del df2
    df3 = pd.np.vstack(df3)
    df4 = pd.np.vstack(df4)
    X_train = df3[:, :3000]
    X_test= df4[:, :3000]
    Y_train= df3[:,3000]
    Y_test= df4[:,3000]
    pat_train=df3[:, 3002:3003]
    pat_test= df4[:, 3002:3003]

    del pd
    return X_train,X_test,Y_train,Y_test,pat_train,pat_test

df2 = pd.read_csv("E:/SleepWell/ASSC/data/lastpurifiedallDataChannel1.csv", header=None)
df2.rename({3000: 'hyp', 3001: 'epoch', 3002: 'patID'}, axis="columns", inplace=True)

trainX, valX, trainY, valY, pat_train, pat_val = patientSplitter('randomizedIDs.csv', df2, .7)
print("Dataframe has been loaded")

#valX -= np.mean(valX, axis=-1, keepdims=True)
#valX /= (np.std(valX, axis=-1,  keepdims=True) + K.epsilon())


[14, 30, 11, 15, 29, 8, 48, 24, 40, 54, 16, 37, 10, 39, 7, 18, 42, 57, 31]
35
21
22
43
9
56
34
58
26
6
3
55
33
2
27
13
46
45
38
47
17
5
25
32
1
52
50
12
41
51
4
19
0
23
53
49
36
20
44
60
59
28
14
30
11
15
29
8
48
24
40
54
16
37
10
39
7
18
42
57
31
Dataframe has been loaded


In [11]:
dummy = trainX
print(dummy)
print(dummy.shape)
dummy -= np.mean(dummy,axis=)
print(dummy)
print(dummy.shape)
print(sum(dummy[0]))

[[  -4.96678944   12.98541056    2.49038056 ...    5.89671056
   -30.74458944  -19.69658944]
 [ -35.80758944  -61.21758944  -78.98558944 ...  -52.10358944
   -37.74158944  -38.29358944]
 [ -50.53858944  -57.99558944  -35.53158944 ...   -7.63658944
  -103.66358944  -60.20458944]
 ...
 [  10.11921056   -7.23368944    1.26010056 ...    7.56191056
    -1.20578944    6.10061056]
 [   6.10061056   -0.10988944    6.83131056 ...   -0.56648944
     3.36071056    9.66251056]
 [  -3.48908944   -1.75378944    2.53873056 ...    1.80808056
   -13.62658944   -9.88258944]]
(82843, 3000)
[[  -4.96678944   12.98541056    2.49038056 ...    5.89671056
   -30.74458944  -19.69658944]
 [ -35.80758944  -61.21758944  -78.98558944 ...  -52.10358944
   -37.74158944  -38.29358944]
 [ -50.53858944  -57.99558944  -35.53158944 ...   -7.63658944
  -103.66358944  -60.20458944]
 ...
 [  10.11921056   -7.23368944    1.26010056 ...    7.56191056
    -1.20578944    6.10061056]
 [   6.10061056   -0.10988944    6.83131056 .

In [6]:
np.unique(trainY)

array([1., 2., 3., 4., 5., 6.])

In [231]:
import random
def epoch_reduction(trainX, trainY, wakeReduction=False, wakeRedSize=0.0, s1Reduction=False, s1RedSize=0.0):
    print("shape of trainX before anything is")
    print(trainX.shape)
    if(wakeReduction):
        print("shape of trainX before wakeReduction is")
        print(trainX.shape)
        wakecount = 0
        trainwakemask = trainY == 6
        print("number of train epochs present in the beginning")
        print(sum(trainwakemask))
        wakecount = sum(trainwakemask)
        willdeletewake = int(wakecount*wakeRedSize)
        my_randoms = random.sample(range(wakecount), willdeletewake)
        print("random list size")
        print(np.shape(my_randoms))
        
        my_randoms = np.sort(my_randoms)
        
        checkercounter = 0
        counter =0
        checker = my_randoms[0]
        for i in range(0, len(trainwakemask)-1):
            if trainwakemask[i]== 1:
                if counter == checker:
                    trainwakemask[i] = 0
                    checkercounter = checkercounter +1
                    checker = my_randoms[checkercounter]
                    print("jhfdskjfdha")
                    print(checker)
                counter = counter + 1        
            
        print("elements in train wake mask")
        print(sum(trainwakemask))
            
        trainwakemask = ~trainwakemask
        trainX = trainX[trainwakemask]
        trainY = trainY[trainwakemask]
        
        print("number of train epochs after filtering is")
        print(sum(trainY==6))
        
        
        
        print("shape of trainX after wakemasking is")
        print(trainX.shape)
    
    if (s1Reduction):
        print("shape of trainX before s1Reduction is")
        print(trainX.shape)
        s1count = 0
        trains1mask = trainY == 2
        
        print("the shape of trains1mask is")
        print(trains1mask.shape)

        s1count = sum(trains1mask)        
        print(s1count)
        willdeletes1 = int(s1count * s1RedSize)
        print("number of s1 epochs to be deleted")
        print(willdeletes1)

        for i in range(willdeletes1, len(trains1mask)):
            trains1mask[i] = 0
        
        
        trains1mask = ~trains1mask
        trainX = trainX[trains1mask]
        trainY = trainY[trains1mask]

    return trainX, trainY


print(trainX.shape)
print(trainY.shape)
dummytrainX, dummytrainY = epoch_reduction(trainX, trainY, wakeReduction=True, wakeRedSize=0.5, s1Reduction=True, s1RedSize=0.0)
print(dummytrainX.shape)
print(dummytrainY.shape)

(73985, 3000)
(73985,)
shape of trainX before anything is
(73985, 3000)
shape of trainX before wakeReduction is
(73985, 3000)
number of train epochs present in the beginning
50286
random list size
(25143,)
jhfdskjfdha
4
jhfdskjfdha
5
jhfdskjfdha
12
jhfdskjfdha
13
jhfdskjfdha
14
jhfdskjfdha
16
jhfdskjfdha
21
jhfdskjfdha
22
jhfdskjfdha
23
jhfdskjfdha
24
jhfdskjfdha
25
jhfdskjfdha
27
jhfdskjfdha
29
jhfdskjfdha
38
jhfdskjfdha
39
jhfdskjfdha
40
jhfdskjfdha
42
jhfdskjfdha
43
jhfdskjfdha
48
jhfdskjfdha
54
jhfdskjfdha
58
jhfdskjfdha
59
jhfdskjfdha
60
jhfdskjfdha
61
jhfdskjfdha
65
jhfdskjfdha
66
jhfdskjfdha
69
jhfdskjfdha
70
jhfdskjfdha
71
jhfdskjfdha
72
jhfdskjfdha
73
jhfdskjfdha
74
jhfdskjfdha
75
jhfdskjfdha
76
jhfdskjfdha
79
jhfdskjfdha
80
jhfdskjfdha
81
jhfdskjfdha
83
jhfdskjfdha
84
jhfdskjfdha
86
jhfdskjfdha
87
jhfdskjfdha
90
jhfdskjfdha
91
jhfdskjfdha
92
jhfdskjfdha
94
jhfdskjfdha
95
jhfdskjfdha
96
jhfdskjfdha
100
jhfdskjfdha
103
jhfdskjfdha
111
jhfdskjfdha
112
jhfdskjfdha
118
jhfdskjfdha

2176
jhfdskjfdha
2177
jhfdskjfdha
2181
jhfdskjfdha
2182
jhfdskjfdha
2183
jhfdskjfdha
2184
jhfdskjfdha
2186
jhfdskjfdha
2187
jhfdskjfdha
2191
jhfdskjfdha
2192
jhfdskjfdha
2193
jhfdskjfdha
2194
jhfdskjfdha
2201
jhfdskjfdha
2202
jhfdskjfdha
2203
jhfdskjfdha
2205
jhfdskjfdha
2206
jhfdskjfdha
2207
jhfdskjfdha
2208
jhfdskjfdha
2209
jhfdskjfdha
2210
jhfdskjfdha
2211
jhfdskjfdha
2212
jhfdskjfdha
2214
jhfdskjfdha
2216
jhfdskjfdha
2219
jhfdskjfdha
2222
jhfdskjfdha
2223
jhfdskjfdha
2224
jhfdskjfdha
2225
jhfdskjfdha
2226
jhfdskjfdha
2229
jhfdskjfdha
2232
jhfdskjfdha
2234
jhfdskjfdha
2238
jhfdskjfdha
2239
jhfdskjfdha
2241
jhfdskjfdha
2242
jhfdskjfdha
2245
jhfdskjfdha
2246
jhfdskjfdha
2247
jhfdskjfdha
2248
jhfdskjfdha
2249
jhfdskjfdha
2251
jhfdskjfdha
2252
jhfdskjfdha
2253
jhfdskjfdha
2254
jhfdskjfdha
2258
jhfdskjfdha
2259
jhfdskjfdha
2260
jhfdskjfdha
2263
jhfdskjfdha
2264
jhfdskjfdha
2265
jhfdskjfdha
2266
jhfdskjfdha
2268
jhfdskjfdha
2269
jhfdskjfdha
2270
jhfdskjfdha
2271
jhfdskjfdha
2276
jhfdskjfd

jhfdskjfdha
4139
jhfdskjfdha
4140
jhfdskjfdha
4141
jhfdskjfdha
4142
jhfdskjfdha
4144
jhfdskjfdha
4145
jhfdskjfdha
4146
jhfdskjfdha
4149
jhfdskjfdha
4150
jhfdskjfdha
4153
jhfdskjfdha
4155
jhfdskjfdha
4157
jhfdskjfdha
4158
jhfdskjfdha
4159
jhfdskjfdha
4162
jhfdskjfdha
4165
jhfdskjfdha
4166
jhfdskjfdha
4167
jhfdskjfdha
4168
jhfdskjfdha
4169
jhfdskjfdha
4171
jhfdskjfdha
4173
jhfdskjfdha
4175
jhfdskjfdha
4176
jhfdskjfdha
4181
jhfdskjfdha
4182
jhfdskjfdha
4183
jhfdskjfdha
4184
jhfdskjfdha
4186
jhfdskjfdha
4187
jhfdskjfdha
4189
jhfdskjfdha
4193
jhfdskjfdha
4194
jhfdskjfdha
4196
jhfdskjfdha
4198
jhfdskjfdha
4201
jhfdskjfdha
4203
jhfdskjfdha
4205
jhfdskjfdha
4208
jhfdskjfdha
4212
jhfdskjfdha
4215
jhfdskjfdha
4217
jhfdskjfdha
4219
jhfdskjfdha
4220
jhfdskjfdha
4223
jhfdskjfdha
4224
jhfdskjfdha
4225
jhfdskjfdha
4227
jhfdskjfdha
4228
jhfdskjfdha
4229
jhfdskjfdha
4230
jhfdskjfdha
4232
jhfdskjfdha
4236
jhfdskjfdha
4237
jhfdskjfdha
4238
jhfdskjfdha
4239
jhfdskjfdha
4242
jhfdskjfdha
4244
jhfdskjfdha
42

6491
jhfdskjfdha
6493
jhfdskjfdha
6498
jhfdskjfdha
6500
jhfdskjfdha
6501
jhfdskjfdha
6502
jhfdskjfdha
6503
jhfdskjfdha
6506
jhfdskjfdha
6508
jhfdskjfdha
6509
jhfdskjfdha
6513
jhfdskjfdha
6515
jhfdskjfdha
6518
jhfdskjfdha
6519
jhfdskjfdha
6520
jhfdskjfdha
6524
jhfdskjfdha
6526
jhfdskjfdha
6527
jhfdskjfdha
6529
jhfdskjfdha
6530
jhfdskjfdha
6531
jhfdskjfdha
6535
jhfdskjfdha
6542
jhfdskjfdha
6543
jhfdskjfdha
6545
jhfdskjfdha
6546
jhfdskjfdha
6547
jhfdskjfdha
6550
jhfdskjfdha
6551
jhfdskjfdha
6552
jhfdskjfdha
6553
jhfdskjfdha
6554
jhfdskjfdha
6555
jhfdskjfdha
6558
jhfdskjfdha
6560
jhfdskjfdha
6563
jhfdskjfdha
6564
jhfdskjfdha
6568
jhfdskjfdha
6570
jhfdskjfdha
6572
jhfdskjfdha
6573
jhfdskjfdha
6576
jhfdskjfdha
6577
jhfdskjfdha
6578
jhfdskjfdha
6580
jhfdskjfdha
6581
jhfdskjfdha
6583
jhfdskjfdha
6585
jhfdskjfdha
6587
jhfdskjfdha
6589
jhfdskjfdha
6590
jhfdskjfdha
6591
jhfdskjfdha
6594
jhfdskjfdha
6595
jhfdskjfdha
6596
jhfdskjfdha
6597
jhfdskjfdha
6598
jhfdskjfdha
6603
jhfdskjfdha
6604
jhfdskjfd

9024
jhfdskjfdha
9026
jhfdskjfdha
9027
jhfdskjfdha
9029
jhfdskjfdha
9030
jhfdskjfdha
9032
jhfdskjfdha
9033
jhfdskjfdha
9037
jhfdskjfdha
9038
jhfdskjfdha
9041
jhfdskjfdha
9042
jhfdskjfdha
9043
jhfdskjfdha
9044
jhfdskjfdha
9045
jhfdskjfdha
9046
jhfdskjfdha
9047
jhfdskjfdha
9048
jhfdskjfdha
9049
jhfdskjfdha
9051
jhfdskjfdha
9052
jhfdskjfdha
9055
jhfdskjfdha
9057
jhfdskjfdha
9058
jhfdskjfdha
9059
jhfdskjfdha
9061
jhfdskjfdha
9063
jhfdskjfdha
9064
jhfdskjfdha
9070
jhfdskjfdha
9071
jhfdskjfdha
9072
jhfdskjfdha
9073
jhfdskjfdha
9074
jhfdskjfdha
9075
jhfdskjfdha
9076
jhfdskjfdha
9077
jhfdskjfdha
9080
jhfdskjfdha
9082
jhfdskjfdha
9083
jhfdskjfdha
9085
jhfdskjfdha
9086
jhfdskjfdha
9088
jhfdskjfdha
9092
jhfdskjfdha
9095
jhfdskjfdha
9096
jhfdskjfdha
9098
jhfdskjfdha
9099
jhfdskjfdha
9101
jhfdskjfdha
9102
jhfdskjfdha
9103
jhfdskjfdha
9104
jhfdskjfdha
9111
jhfdskjfdha
9112
jhfdskjfdha
9113
jhfdskjfdha
9114
jhfdskjfdha
9115
jhfdskjfdha
9117
jhfdskjfdha
9121
jhfdskjfdha
9122
jhfdskjfdha
9124
jhfdskjfd

...

40090
jhfdskjfdha
40094
jhfdskjfdha
40095
jhfdskjfdha
40097
jhfdskjfdha
40099
jhfdskjfdha
40101
jhfdskjfdha
40102
jhfdskjfdha
40104
jhfdskjfdha
40105
jhfdskjfdha
40108
jhfdskjfdha
40109
jhfdskjfdha
40110
jhfdskjfdha
40111
jhfdskjfdha
40112
jhfdskjfdha
40113
jhfdskjfdha
40115
jhfdskjfdha
40116
jhfdskjfdha
40117
jhfdskjfdha
40119
jhfdskjfdha
40120
jhfdskjfdha
40126
jhfdskjfdha
40127
jhfdskjfdha
40128
jhfdskjfdha
40131
jhfdskjfdha
40132
jhfdskjfdha
40138
jhfdskjfdha
40139
jhfdskjfdha
40140
jhfdskjfdha
40142
jhfdskjfdha
40143
jhfdskjfdha
40145
jhfdskjfdha
40148
jhfdskjfdha
40150
jhfdskjfdha
40151
jhfdskjfdha
40153
jhfdskjfdha
40155
jhfdskjfdha
40156
jhfdskjfdha
40157
jhfdskjfdha
40158
jhfdskjfdha
40159
jhfdskjfdha
40163
jhfdskjfdha
40165
jhfdskjfdha
40166
jhfdskjfdha
40167
jhfdskjfdha
40168
jhfdskjfdha
40169
jhfdskjfdha
40170
jhfdskjfdha
40171
jhfdskjfdha
40172
jhfdskjfdha
40175
jhfdskjfdha
40176
jhfdskjfdha
40178
jhfdskjfdha
40181
jhfdskjfdha
40182
jhfdskjfdha
40185
jhfdskjfdha
40188
jhfd

jhfdskjfdha
42561
jhfdskjfdha
42563
jhfdskjfdha
42565
jhfdskjfdha
42571
jhfdskjfdha
42573
jhfdskjfdha
42574
jhfdskjfdha
42579
jhfdskjfdha
42582
jhfdskjfdha
42585
jhfdskjfdha
42588
jhfdskjfdha
42589
jhfdskjfdha
42591
jhfdskjfdha
42592
jhfdskjfdha
42593
jhfdskjfdha
42594
jhfdskjfdha
42597
jhfdskjfdha
42599
jhfdskjfdha
42601
jhfdskjfdha
42605
jhfdskjfdha
42606
jhfdskjfdha
42608
jhfdskjfdha
42609
jhfdskjfdha
42610
jhfdskjfdha
42615
jhfdskjfdha
42616
jhfdskjfdha
42618
jhfdskjfdha
42619
jhfdskjfdha
42622
jhfdskjfdha
42623
jhfdskjfdha
42625
jhfdskjfdha
42626
jhfdskjfdha
42628
jhfdskjfdha
42629
jhfdskjfdha
42630
jhfdskjfdha
42632
jhfdskjfdha
42634
jhfdskjfdha
42636
jhfdskjfdha
42638
jhfdskjfdha
42639
jhfdskjfdha
42641
jhfdskjfdha
42644
jhfdskjfdha
42646
jhfdskjfdha
42649
jhfdskjfdha
42650
jhfdskjfdha
42652
jhfdskjfdha
42653
jhfdskjfdha
42658
jhfdskjfdha
42660
jhfdskjfdha
42661
jhfdskjfdha
42663
jhfdskjfdha
42665
jhfdskjfdha
42667
jhfdskjfdha
42669
jhfdskjfdha
42672
jhfdskjfdha
42679
jhfdskjfdh

jhfdskjfdha
45038
jhfdskjfdha
45041
jhfdskjfdha
45043
jhfdskjfdha
45047
jhfdskjfdha
45048
jhfdskjfdha
45050
jhfdskjfdha
45051
jhfdskjfdha
45052
jhfdskjfdha
45053
jhfdskjfdha
45056
jhfdskjfdha
45061
jhfdskjfdha
45063
jhfdskjfdha
45064
jhfdskjfdha
45065
jhfdskjfdha
45069
jhfdskjfdha
45070
jhfdskjfdha
45073
jhfdskjfdha
45075
jhfdskjfdha
45076
jhfdskjfdha
45077
jhfdskjfdha
45079
jhfdskjfdha
45080
jhfdskjfdha
45083
jhfdskjfdha
45084
jhfdskjfdha
45086
jhfdskjfdha
45087
jhfdskjfdha
45088
jhfdskjfdha
45092
jhfdskjfdha
45093
jhfdskjfdha
45096
jhfdskjfdha
45098
jhfdskjfdha
45099
jhfdskjfdha
45103
jhfdskjfdha
45106
jhfdskjfdha
45111
jhfdskjfdha
45113
jhfdskjfdha
45116
jhfdskjfdha
45120
jhfdskjfdha
45122
jhfdskjfdha
45125
jhfdskjfdha
45129
jhfdskjfdha
45130
jhfdskjfdha
45131
jhfdskjfdha
45133
jhfdskjfdha
45134
jhfdskjfdha
45135
jhfdskjfdha
45140
jhfdskjfdha
45143
jhfdskjfdha
45147
jhfdskjfdha
45148
jhfdskjfdha
45153
jhfdskjfdha
45154
jhfdskjfdha
45156
jhfdskjfdha
45158
jhfdskjfdha
45159
jhfdskjfdh

47580
jhfdskjfdha
47583
jhfdskjfdha
47585
jhfdskjfdha
47586
jhfdskjfdha
47592
jhfdskjfdha
47593
jhfdskjfdha
47594
jhfdskjfdha
47595
jhfdskjfdha
47596
jhfdskjfdha
47598
jhfdskjfdha
47602
jhfdskjfdha
47604
jhfdskjfdha
47605
jhfdskjfdha
47606
jhfdskjfdha
47607
jhfdskjfdha
47609
jhfdskjfdha
47610
jhfdskjfdha
47612
jhfdskjfdha
47615
jhfdskjfdha
47616
jhfdskjfdha
47621
jhfdskjfdha
47624
jhfdskjfdha
47625
jhfdskjfdha
47626
jhfdskjfdha
47628
jhfdskjfdha
47632
jhfdskjfdha
47634
jhfdskjfdha
47635
jhfdskjfdha
47636
jhfdskjfdha
47637
jhfdskjfdha
47638
jhfdskjfdha
47640
jhfdskjfdha
47641
jhfdskjfdha
47643
jhfdskjfdha
47644
jhfdskjfdha
47645
jhfdskjfdha
47647
jhfdskjfdha
47648
jhfdskjfdha
47650
jhfdskjfdha
47651
jhfdskjfdha
47652
jhfdskjfdha
47653
jhfdskjfdha
47654
jhfdskjfdha
47655
jhfdskjfdha
47656
jhfdskjfdha
47658
jhfdskjfdha
47660
jhfdskjfdha
47662
jhfdskjfdha
47663
jhfdskjfdha
47664
jhfdskjfdha
47666
jhfdskjfdha
47667
jhfdskjfdha
47668
jhfdskjfdha
47669
jhfdskjfdha
47670
jhfdskjfdha
47671
jhfd

50118
jhfdskjfdha
50119
jhfdskjfdha
50120
jhfdskjfdha
50121
jhfdskjfdha
50122
jhfdskjfdha
50123
jhfdskjfdha
50127
jhfdskjfdha
50129
jhfdskjfdha
50130
jhfdskjfdha
50133
jhfdskjfdha
50135
jhfdskjfdha
50138
jhfdskjfdha
50139
jhfdskjfdha
50140
jhfdskjfdha
50143
jhfdskjfdha
50144
jhfdskjfdha
50145
jhfdskjfdha
50147
jhfdskjfdha
50148
jhfdskjfdha
50149
jhfdskjfdha
50150
jhfdskjfdha
50152
jhfdskjfdha
50153
jhfdskjfdha
50154
jhfdskjfdha
50157
jhfdskjfdha
50158
jhfdskjfdha
50159
jhfdskjfdha
50160
jhfdskjfdha
50165
jhfdskjfdha
50167
jhfdskjfdha
50168
jhfdskjfdha
50171
jhfdskjfdha
50172
jhfdskjfdha
50173
jhfdskjfdha
50176
jhfdskjfdha
50181
jhfdskjfdha
50182
jhfdskjfdha
50183
jhfdskjfdha
50184
jhfdskjfdha
50185
jhfdskjfdha
50187
jhfdskjfdha
50188
jhfdskjfdha
50189
jhfdskjfdha
50190
jhfdskjfdha
50191
jhfdskjfdha
50192
jhfdskjfdha
50193
jhfdskjfdha
50195
jhfdskjfdha
50196
jhfdskjfdha
50197
jhfdskjfdha
50200
jhfdskjfdha
50206
jhfdskjfdha
50207
jhfdskjfdha
50209
jhfdskjfdha
50215
jhfdskjfdha
50217
jhfd

In [265]:
mask = trainY == 6
wakeidx = np.nonzero(mask)[0]
dropidx = np.random.choice(wakeidx,size=int(wakeidx.shape[0]*.1),replace=False)
trainX_ = np.delete(trainX,chosen,axis=0)
trainY_ = np.delete(trainY,chosen,axis=0)

mask = trainY == 2
wakeidx = np.nonzero(mask)[0]
dropidx = np.random.choice(wakeidx,size=int(wakeidx.shape[0]*.1),replace=False)
trainX_ = np.delete(trainX,chosen,axis=0)
trainY_ = np.delete(trainY,chosen,axis=0)

In [232]:
sum(dummytrainY==6)

25142

In [228]:
import random
trainwakemask = dummytrainY == 2
sum(trainwakemask)
my_randoms = random.sample(range(100), 2)
print(my_randoms)
your_randoms = np.sort(my_randoms)
print(your_randoms)
np.shape(your_randoms)
#for 
#for delete in my_randoms

[79, 13]
[13 79]


(2,)

In [78]:
indexlist = list(range(len(trainwakemask)))
#trainX.type
indexlist = np.asarray(indexlist)
indexlist[trainwakemask].shape

(2727,)

In [37]:
trainX /= np.std(trainX,axis=1,keepdims=True)

C:\Users\User\Anaconda3\envs\thesis\lib\site-packages\numpy\core\_methods.py:112: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [ ]:
mean = np.mean(trainX)
std = np.std(trainX)

In [ ]:
valY2 = valY - mean
valY2 = valY2/std

In [ ]:
print(np.mean(valY2))


In [ ]:
print(np.unique(valY))
print(np.unique(predY))

In [ ]:
for i in range (1,len(valY)+1):
    if int(valY[i-1])==6:
        valY[i-1]=5
for j in range (1,len(predY)+1):
    if int(predY[j-1])==6:
        predY[j-1]=5    

In [ ]:
for j in range (1,len(predY)+1):
    if int(predY[j-1])==4:
        print(j)

In [ ]:
import numpy as np
#predY=predY-1
print(np.unique(predY))
print(np.unique(valY))

trueY=trueY+1

In [ ]:
#confMat = confusion_matrix(valY,predY)
print(confMat)
print(np.unique(pat_val))
print(np.unique(predY))

In [28]:
trueX = valX
trueX = np.expand_dims(trueX, axis=-1)
predY = model.predict(trueX, verbose=0)    
predY = np.argmax(predY[0], axis=-1)

In [42]:
print(np.unique(predY+1))
accuracy_score(valY-1 , predY)

[2 3 4 5 6]


0.009340812163916188

In [43]:
# trueY= valY
# trueX = np.expand_dims(valX, axis=-1)
# predY = model.predict(trueX, verbose=0)    
# predY = np.argmax(predY, axis=-1)
# #



# trueY = valY
# #trueX = np.expand_dims(trueX, axis=0)
# #trueX = trueX.T
# #trueX = np.expand_dims(trueX, axis=0)
# trueX = valX
# trueX = np.expand_dims(trueX, axis=-1)
# predY = model.predict(trueX, verbose=0)    
# predY = np.argmax(predY, axis=-1)



#predY=predY+1

from collections import Counter

#confMat = confusion_matrix(trueY,predY)[1:,:-1]
confMat = confusion_matrix(trueY, predY)
match = sum(predY==trueY-1)

sens= []
spec= []

for each in set(trueY-1):
    each = int(each)
    sens.append( confMat[each,each]/sum(confMat[each,:]) )
    spec.append( (match - confMat[each,each])/((match - confMat[each,each]+sum(confMat[:,each])-confMat[each,each])))

acc = match/len(trueY)

patAcc = []
for pat in np.unique(pat_val).astype(int):
    mask = pat_val[:,0] == pat
    patAcc.append(accuracy_score(valY[mask] -1 , predY[mask]))
patAccAvg = np.mean(patAcc)


print("model specificity is {}".format(spec))

print("model sensitivity is")
print(sens)
print("model accuracy is")
print(acc)
print("mean sensitivity is {}" .format(np.mean(sens)))
print("mean specificity is {}" .format(np.mean(spec)))
print("patient accuracy is")
print(patAccAvg)



patsens = []
patspec = []

dummyvalY = valY -1

for each in np.unique(dummyvalY).astype(int):
    each = int(each)
    patsens.append(confMat[each, each] / sum(confMat[each, :]))
    patspec.append((match - confMat[each, each]) / ((match - confMat[each, each] + sum(confMat[:, each]) - confMat[each, each])))

acc = match / len(valY)

patAcc = []
for pat in np.unique(pat_val).astype(int):
    mask = pat_val[:, 0] == pat
    patAcc.append(accuracy_score(valY[mask] - 1, predY[mask]))
patAccAvg = np.mean(patAcc)
print("patient accuracy is")
print(patAccAvg)

perClassSens= []
perClassSpec = []
correctedtrueY = valY - 1
print(correctedtrueY.shape)
print(np.unique(correctedtrueY))
for each_class in np.unique(correctedtrueY):
    print("fjskdjfh")
    each_class= int(each_class)
    mask = correctedtrueY == each_class
    totalcandidatesforthatclass = sum(mask)
    print(totalcandidatesforthatclass)
    match = sum(correctedtrueY[mask]==predY[mask])
    
    print(match)
    acc = match/totalcandidatesforthatclass
    print(acc)
    
    
#     tp = match
#     tn = sum(correctedtrueY[~mask] - sum(correctedtrueY[~mask]== each_class) 
#     fp = sum([])
#     fn = 
    
    #perClassSens.append(match/totalcandidatesforthatclass)
    #perClassSens.append()
    
    #confMat = confusion_matrix(correctedtrueY[mask], predY[mask])
    #print(confMat)
    print("done")
    #perClassSens.append(confMat[each_class, each_class] / sum(confMat[each_class, :]))
    #perClassSpec.append((match-confMat[each_class, each_class])/ ((match - confMat[each_class, each_class] + sum(confMat[:, each_class]) - confMat[each_class, each_class])))
print(perClassSens)
print(perClassSpec)
print(match)

# mask = correctedtrueY ==1
# total = sum(mask)
# print(total)
# match = sum(correctedtrueY==predY)
# print(match)

model specificity is [0.9888380080137379, 0.907554747291051, 0.955816220635869, 0.8927725625588495, 0.5390654657141871, 1.0]
model sensitivity is
[0.14919663351185922, 0.940754039497307, 0.5276796230859835, 0.0007668711656441718, 0.013052743740010656, 0.0]
model accuracy is
0.7252456888559243
mean sensitivity is 0.2719083185001341
mean specificity is 0.8806745007022826
patient accuracy is
0.6561217208194843
patient accuracy is
0.6561217208194843
(43144,)
[0. 1. 2. 3. 4. 5.]
fjskdjfh
1307
0
0.0
done
fjskdjfh
7798
45
0.005770710438573993
done
fjskdjfh
1698
792
0.4664310954063604
done
fjskdjfh
1304
1192
0.9141104294478528
done
fjskdjfh
3754
2740
0.7298881193393714
done
fjskdjfh
27283
26521
0.9720705201040941
done
[]
[]
26521


In [ ]:
from collections import Counter

In [ ]:
import matplotlib.pyplot as plt
c = 1

#dummyvalY = valY
#dummypredY = predY+1
#true_hypnogram = valY[1+ (c-1)*3000:c*3000]
#predicted_hypnogram = dummypredY[1+ (c-1)*3000:c*3000]
true_hypnogram.shape
f= plt.figure(figsize=(12,3))
plt.plot(true_hypnogram,scalex=True)

plt.yticks([1, 2, 3, 4, 5, 6], ['S1', 'S2', 'S3', 'S4', 'REM', 'W'])
plt.xlim((1000,1800))
plt.show()


from sklearn.metrics import accuracy_score
print (accuracy_score(true_hypnogram, predicted_hypnogram))
f.savefig("E:/SleepWell/fig/truehyp.pdf")

In [ ]:
performance1 = true_hypnogram
performance2 = predicted_hypnogram

f, axes = plt.subplots(1, 2)    
plt.plot(predicted_hypnogram)
axes[0].set_yticks([1, 2, 3, 4, 5, 6], ['S1', 'S2', 'S3', 'S4', 'REM', 'W'])
axes[0].set_xticks((1000,1800))
axes[0].grid(linewidth=.2)

plt.plot(predicted_hypnogram)
axes[1].set_yticks([1, 2, 3, 4, 5, 6], ['S1', 'S2', 'S3', 'S4', 'REM', 'W'])
axes[1].set_xticks((1000,1800))
axes[1].grid(linewidth=.2)
plt.ylabel('Accuracy')



# sns.barplot(y_pos2, performance2, color='teal',  orient='v' , ax=axes[1])
# plt.xticks(y_pos2, objects2, rotation=60, fontsize=8)
# # plt.ylabel('Accuracy')
# # plt.title('Per Patient Accuracy')
# axes[1].set(ylim=[.4,.99],xlabel="(b)")
plt.xlim(1000,2000)
plt.ylim(.4,.99)
# plt.figure(figsize=(3,8))
# # plt.xlabel('(b)')
# axes[1].grid(linewidth=.2)

f.tight_layout()

#f.savefig("E:/SleepWell/fig/PerPatAccFinal4.pdf")
plt.show()




In [ ]:
#true_hypnogram = valY[1:3000]
#predicted_hypnogram = predY[1:3000]
true_hypnogram.shape
f=plt.figure(figsize=(12,3))
plt.plot(predicted_hypnogram,scalex=True)

plt.yticks([1, 2, 3, 4, 5, 6], ['S1', 'S2', 'S3', 'S4', 'REM', 'W'])
plt.xlim((1000,1800))
plt.show()
f.savefig("E:/SleepWell/fig/predhyp.pdf")

In [ ]:
import pandas as pd
#patAccDir = "E:\SleepWell\ASSC\logs\lastAlldataChannel1_2019-01-19 18-37-49.652298/epoch22patAcc.csv"
#patAccDir = "E:/SleepWell/ASSC/logs/EEGcasetteonlyADGC_2019-01-21 14-52-02.119827/epoch8patAcc.csv"
#df = pd.read_csv(patAccDir,header=None)
from numpy import genfromtxt
my_data = patAcc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set(style='dark')
print(my_data)
your_data= my_data
print(your_data)
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

#objects = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19')
#objects = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19')
pat_valu = np.unique(pat_val)



data = pd.read_csv("patientnames.csv", header=None, delimiter=' ') 
a= data.iloc[:,0].values
name_val=[]
patient_list = np.unique(pat_valu)
for i in range (1,len(np.unique(pat_valu))+1):
    patHere = patient_list[i-1]
    name_val.append(a[ int(patHere) -1 ])
objects=tuple(name_val)  




f = plt.figure()

y_pos = np.arange(len(objects))
performance = your_data
 
sns.barplot(y_pos, performance,
#             align='center',
            alpha=1,
            color='teal'
           )
plt.xticks(y_pos, objects, rotation=60, fontsize=8)
plt.ylabel('Accuracy')
plt.title('Per Patient Accuracy')
plt.xlim(-.5,11.5)
plt.ylim(.2,.99)
plt.grid(linewidth=.2)
f.savefig("E:/SleepWell/fig/SleepCasettepat.pdf")
plt.show()

In [ ]:
# import numpy as np
# print(np.unique(pat_val))
# print(np.unique(pat_train))
# print(pat_val)
# pat_val.shape
np.unique(pat_val)

In [ ]:
data = pd.read_csv("patientnames.csv", header=None, delimiter=' ') 
a= data.iloc[:,0].values
name_val=[]
patient_list = np.unique(pat_val)
for i in range (1,len(np.unique(pat_val))+1):
    patHere = patient_list[i-1]
    name_val.append(a[ int(patHere) ])
name_val=tuple(name_val)  



# int(pat_val[3])
# print(a[int(pat_val[3])])
# len(pat_val)
# a

In [ ]:
name_val=[]
patient_list = np.unique(pat_val)
for i in range (1,len(np.unique(pat_val))+1):
    patHere = patient_list[i-1]
    name_val.append(a[ int(patHere) ])
name_val=tuple(name_val)   

In [ ]:
name_val=tuple(name_val)

name_val

In [ ]:
name_val

In [ ]:
objects = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19')

In [ ]:
objects

In [ ]:
########### Per Patient Plot #############################


import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt



#objects = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19')

data = pd.read_csv("patientnames.csv", header=None, delimiter=' ') 
a= data.iloc[:,0].values
name_val=[]
patient_list = np.unique(pat_val)
for i in range (1,len(np.unique(pat_val))+1):
    patHere = patient_list[i-1]
    name_val.append(a[ int(patHere) ])
objects=tuple(name_val)  


y_pos = np.arange(len(objects))
performance = your_data



plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Accuracy')
plt.title('Per Patient Accuracy')
 
plt.show()
f = plt.figure()
f.savefig("E:/SleepWell/fig/PerPatAccTrial.pdf")

In [ ]:
SC4001 1
SC4002 2
SC4011 3
SC4012 4
SC4021 5
SC4022 6
SC4031 7
SC4032 8
SC4041 9
SC4042 10
SC4051 11
SC4052 12
SC4061 13
SC4062 14
SC4071 15
SC4072 16
SC4081 17
SC4082 18
SC4091 19
SC4092 20
SC4101 21
SC4102 22
SC4111 23
SC4112 24
SC4121 25
SC4122 26
SC4131 27
SC4141 28
SC4142 29
SC4151 30
SC4152 31
SC4161 32
SC4162 33
SC4171 34
SC4172 35
SC4181 36
SC4182 37
SC4191 38
SC4192 39
ST7011 40
ST7022 41
ST7041 42
ST7052 43
ST7061 44
ST7071 45
ST7082 46
ST7092 47
ST7101 48
ST7112 49
ST7121 50
ST7132 51
ST7141 52
ST7151 53
ST7162 54
ST7171 55
ST7182 56
ST7192 57
ST7201 58
ST7212 59
ST7221 60
ST7241 61

In [ ]:
trueY= valY
trueX = np.expand_dims(valX, axis=-1)
predY = model.predict(trueX, verbose=0)    
predY = np.argmax(predY, axis=-1)

confMat = confusion_matrix(trueY,predY)[1:,:-1]
match = sum(predY==trueY-1)

In [ ]:
match

In [ ]:
confMat
confusion_matrix(trueY,predY)
#np.mean(valX, axis=-1, keepdims=True)

In [ ]:
from matplotlib import pyplot as plt

plt.hist(valY)
plt.show()

In [ ]:
### Normalization #####

# from sklearn.preprocessing import StandardScaler
# data = valX.flatten('A')
# data = np.expand_dims(data, axis=1)



# scaler = StandardScaler()
# outdata = scaler.fit_transform(data)
# toutdata = outdata.reshape(int(outdata.shape[0]/3000),3000)

trueX = np.expand_dims(valX, axis=-1)
predY = model.predict(trueX, verbose=0)



In [15]:
predY.shape
    

AttributeError: 'list' object has no attribute 'shape'

In [17]:
trueY = valY
trueX = np.expand_dims(valX, axis=0)
predY = model.predict(trueX, verbose=0)


TypeError: predict() got an unexpected keyword argument 'axis'

In [ ]:
patAcc

In [ ]:
confMat = confusion_matrix(trueY,predY)[1:,:-1]
match = sum(predY==trueY-1)

sens= []
spec= []

for each in set(trueY-1):
    each = int(each)
    sens.append( confMat[each,each]/sum(confMat[each,:]) )
    spec.append( (match - confMat[each,each])/((match - confMat[each,each]+sum(confMat[:,each])-confMat[each,each])))

acc = match/len(trueY)

patAcc = []
for pat in np.unique(pat_val).astype(int):
    mask = pat_val[:,0] == pat
    patAcc.append(accuracy_score(valY[mask]-1, predY[mask]))
patAccAvg = np.mean(patAcc)


print("model specificity is {}".format(spec))

print("model sensitivity is")
print(sens)
print("model accuracy is")
print(acc)
print("patient accuracy is")
print(patAccAvg)

In [ ]:
patAcc

In [ ]:
#mask= pat_val ==1
#accuracy_score(data[mask[:,0]], predY[mask[:,0]])
sum(confMat[each,:])

In [ ]:
sens.shape

In [ ]:
patAcc = []
for pat in np.unique(pat_val).astype(int):
    mask = pat_val == pat
    data = np.expand_dims(valY, axis=-1)
    patAcc.append(accuracy_score(data[mask[:,0]], predY[mask[:,0]]))
patAccAvg = np.mean(patAcc)

In [ ]:
patAcc

In [ ]:
predY[1905,:]

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

class_sensitivity=np.zeros(6)
class_specificity= np.zeros(6)
class_accuracy = np.zeros(6)


trueY = valY
#trueX = np.expand_dims(trueX, axis=0)
#trueX = trueX.T
#trueX = np.expand_dims(trueX, axis=0)
trueX = valX
trueX = np.expand_dims(trueX, axis=-1)
predY = model.predict(trueX, verbose=0)    
predY = np.argmax(predY, axis=-1)
confMat = confusion_matrix(trueY,predY)[1:,:-1]

# trueY = trueY.astype(int)

#predY = np.expand_dims(predY, axis=1)



# patAcc = []
# #cm1=np.zeros((6, 6))
# class_sensitivity=np.zeros(6)
# class_specificity= np.zeros(6)
# class_accuracy = np.zeros(6)
# print("printing the shape of predY")
# print(predY.shape)

    
# for class_count in np.unique(trueY).astype(int):
#     mask = valY == class_count
#     cm1 = confusion_matrix(trueY[mask], predY[mask])
#     total=sum(sum(cm1))
    
    
#     tp = trueY == predY
#     tp_bins = trueY[tp]
    
#     tp_sum = np.bincount(tp_bins)
#     #predY2 = np.expand_dims(predY, axis=1)
    
    
#     pred_sum = np.bincount(predY)
#     true_sum = np.bincount(trueY)
    
    
#     fp = pred_sum - tp_sum
#     fn = true_sum - tp_sum
    
#     tp = tp_sum
#     tp = np.array(tp)
#     fp = np.array(fp)
#     fn = np.array(fn)
        
#     tn = y_true.shape[0] - tp - fp - fn
    

    
#     class_sensitivity[class_count] = class_sensitivity[class_count]+ tp/(tp+fn)
#     class_specificity[class_count] = class_specificity[class_count]+ tn/(tn+fp)
#     class_accuracy[class_count]= class_accuracy[class_count]+ (tp+tn)/(tp+tn+fp+fn)
#     f1_score(trueY, predY, average='macro') 
    
# Accuracy = np.mean(class_accuracy)
# Specificity= np.mean(class_specificity)
# Sensitivity= np.mean(class_sensitivity)


# print(Accuracy)
# print(Specificity)
# print(Sensitivity)

In [ ]:
def _check_targets(y_true, y_pred):
    """Check that y_true and y_pred belong to the same classification task
    This converts multiclass or binary types to a common shape, and raises a
    ValueError for a mix of multilabel and multiclass targets, a mix of
    multilabel formats, for the presence of continuous-valued or multioutput
    targets, or for targets of different lengths.
    Column vectors are squeezed to 1d, while multilabel formats are returned
    as CSR sparse label indicators.
    Parameters
    ----------
    y_true : array-like
    y_pred : array-like
    Returns
    -------
    type_true : one of {'multilabel-indicator', 'multiclass', 'binary'}
        The type of the true target data, as output by
        ``utils.multiclass.type_of_target``
    y_true : array or indicator matrix
    y_pred : array or indicator matrix
    """
    check_consistent_length(y_true, y_pred)
    type_true = type_of_target(y_true)
    type_pred = type_of_target(y_pred)

    y_type = {type_true, type_pred}
    if y_type == {"binary", "multiclass"}:
        y_type = {"multiclass"}

    if len(y_type) > 1:
        raise ValueError("Classification metrics can't handle a mix of {0} "
                         "and {1} targets".format(type_true, type_pred))

    # We can't have more than one value on y_type => The set is no more needed
    y_type = y_type.pop()

    # No metrics support "multiclass-multioutput" format
    if (y_type not in ["binary", "multiclass", "multilabel-indicator"]):
        raise ValueError("{0} is not supported".format(y_type))

    if y_type in ["binary", "multiclass"]:
        y_true = column_or_1d(y_true)
        y_pred = column_or_1d(y_pred)
        if y_type == "binary":
            unique_values = np.union1d(y_true, y_pred)
            if len(unique_values) > 2:
                y_type = "multiclass"

    if y_type.startswith('multilabel'):
        y_true = csr_matrix(y_true)
        y_pred = csr_matrix(y_pred)
        y_type = 'multilabel-indicator'

    return y_type, y_true, y_pred



In [ ]:
def multilabel_confusion_matrix(y_true, y_pred, sample_weight=None,
                                labels=None, samplewise=False):
    """Compute a confusion matrix for each class or sample
    .. versionadded:: 0.21
    Compute class-wise (default) or sample-wise (samplewise=True) multilabel
    confusion matrix to evaluate the accuracy of a classification, and output
    confusion matrices for each class or sample.
    In multilabel confusion matrix :math:`MCM`, the count of true negatives
    is :math:`MCM_{:,0,0}`, false negatives is :math:`MCM_{:,1,0}`,
    true positives is :math:`MCM_{:,1,1}` and false positives is
    :math:`MCM_{:,0,1}`.
    Multiclass data will be treated as if binarized under a one-vs-rest
    transformation. Returned confusion matrices will be in the order of
    sorted unique labels in the union of (y_true, y_pred).
    Read more in the :ref:`User Guide <multilabel_confusion_matrix>`.
    Parameters
    ----------
    y_true : 1d array-like, or label indicator array / sparse matrix
        of shape (n_samples, n_outputs) or (n_samples,)
        Ground truth (correct) target values.
    y_pred : 1d array-like, or label indicator array / sparse matrix
        of shape (n_samples, n_outputs) or (n_samples,)
        Estimated targets as returned by a classifier
    sample_weight : array-like of shape = (n_samples,), optional
        Sample weights
    labels : array-like
        A list of classes or column indices to select some (or to force
        inclusion of classes absent from the data)
    samplewise : bool, default=False
        In the multilabel case, this calculates a confusion matrix per sample
    Returns
    -------
    multi_confusion : array, shape (n_outputs, 2, 2)
        A 2x2 confusion matrix corresponding to each output in the input.
        When calculating class-wise multi_confusion (default), then
        n_outputs = n_labels; when calculating sample-wise multi_confusion
        (samplewise=True), n_outputs = n_samples. If ``labels`` is defined,
        the results will be returned in the order specified in ``labels``,
        otherwise the results will be returned in sorted order by default.
    See also
    --------
    confusion_matrix
    Notes
    -----
    The multilabel_confusion_matrix calculates class-wise or sample-wise
    multilabel confusion matrices, and in multiclass tasks, labels are
    binarized under a one-vs-rest way; while confusion_matrix calculates
    one confusion matrix for confusion between every two classes.
    Examples
    --------
    Multilabel-indicator case:
    >>> import numpy as np
    >>> from sklearn.metrics import multilabel_confusion_matrix
    >>> y_true = np.array([[1, 0, 1],
    ...                    [0, 1, 0]])
    >>> y_pred = np.array([[1, 0, 0],
    ...                    [0, 1, 1]])
    >>> multilabel_confusion_matrix(y_true, y_pred)
    array([[[1, 0],
            [0, 1]],
    <BLANKLINE>
           [[1, 0],
            [0, 1]],
    <BLANKLINE>
           [[0, 1],
            [1, 0]]])
    Multiclass case:
    >>> y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
    >>> y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
    >>> multilabel_confusion_matrix(y_true, y_pred,
    ...                             labels=["ant", "bird", "cat"])
    array([[[3, 1],
            [0, 2]],
    <BLANKLINE>
           [[5, 0],
            [1, 0]],
    <BLANKLINE>
           [[2, 1],
            [1, 2]]])
    """
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
    if sample_weight is not None:
        sample_weight = column_or_1d(sample_weight)
    check_consistent_length(y_true, y_pred, sample_weight)

    if y_type not in ("binary", "multiclass", "multilabel-indicator"):
        raise ValueError("%s is not supported" % y_type)

    present_labels = unique_labels(y_true, y_pred)
    if labels is None:
        labels = present_labels
        n_labels = None
    else:
        n_labels = len(labels)
        labels = np.hstack([labels, np.setdiff1d(present_labels, labels,
                                                 assume_unique=True)])

    if y_true.ndim == 1:
        if samplewise:
            raise ValueError("Samplewise metrics are not available outside of "
                             "multilabel classification.")

        le = LabelEncoder()
        le.fit(labels)
        y_true = le.transform(y_true)
        y_pred = le.transform(y_pred)
        sorted_labels = le.classes_

        # labels are now from 0 to len(labels) - 1 -> use bincount
        tp = y_true == y_pred
        tp_bins = y_true[tp]
        if sample_weight is not None:
            tp_bins_weights = np.asarray(sample_weight)[tp]
        else:
            tp_bins_weights = None

        if len(tp_bins):
            tp_sum = np.bincount(tp_bins, weights=tp_bins_weights,
                                 minlength=len(labels))
        else:
            # Pathological case
            true_sum = pred_sum = tp_sum = np.zeros(len(labels))
        if len(y_pred):
            pred_sum = np.bincount(y_pred, weights=sample_weight,
                                   minlength=len(labels))
        if len(y_true):
            true_sum = np.bincount(y_true, weights=sample_weight,
                                   minlength=len(labels))

        # Retain only selected labels
        indices = np.searchsorted(sorted_labels, labels[:n_labels])
        tp_sum = tp_sum[indices]
        true_sum = true_sum[indices]
        pred_sum = pred_sum[indices]

    else:
        sum_axis = 1 if samplewise else 0

        # All labels are index integers for multilabel.
        # Select labels:
        if not np.array_equal(labels, present_labels):
            if np.max(labels) > np.max(present_labels):
                raise ValueError('All labels must be in [0, n labels) for '
                                 'multilabel targets. '
                                 'Got %d > %d' %
                                 (np.max(labels), np.max(present_labels)))
            if np.min(labels) < 0:
                raise ValueError('All labels must be in [0, n labels) for '
                                 'multilabel targets. '
                                 'Got %d < 0' % np.min(labels))

        if n_labels is not None:
            y_true = y_true[:, labels[:n_labels]]
            y_pred = y_pred[:, labels[:n_labels]]

        # calculate weighted counts
        true_and_pred = y_true.multiply(y_pred)
        tp_sum = count_nonzero(true_and_pred, axis=sum_axis,
                               sample_weight=sample_weight)
        pred_sum = count_nonzero(y_pred, axis=sum_axis,
                                 sample_weight=sample_weight)
        true_sum = count_nonzero(y_true, axis=sum_axis,
                                 sample_weight=sample_weight)

    fp = pred_sum - tp_sum
    fn = true_sum - tp_sum
    tp = tp_sum

    if sample_weight is not None and samplewise:
        sample_weight = np.array(sample_weight)
        tp = np.array(tp)
        fp = np.array(fp)
        fn = np.array(fn)
        tn = sample_weight * y_true.shape[1] - tp - fp - fn
    elif sample_weight is not None:
        tn = sum(sample_weight) - tp - fp - fn
    elif samplewise:
        tn = y_true.shape[1] - tp - fp - fn
    else:
        tn = y_true.shape[0] - tp - fp - fn

    return np.array([tn, fp, fn, tp]).T.reshape(-1, 2, 2)



In [ ]:
predY.shape

In [ ]:
from sklearn.metrics import confusion_matrix

cm1 = confusion_matrix(trueY,predY)
print('Confusion Matrix : \n', cm1)

total1=sum(sum(cm1))
#####from confusion matrix calculate accuracy
accuracy1=(cm1[0,0]+cm1[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)



In [ ]:
import numpy as np
jhjh=np.zeros((2,6))
jhjh[1,1]=3
print(jhjh)
trueX.shape

In [ ]:
কাজ টা করতে হলে আমার আগে একটা ট্রুথ এরে লাগবে। 
ট্রুথ এর বানাতেে হলে আগে প্রেডিকশন আর অরিজিনাল ওয়াই লাগবে
প্রেডিকশন আর অরিজিনাল ওয়াই আনার আগে ডাটা নরমালাইজ করতে হবে। 

In [ ]:
#confusion_matrix(trueY[mask], predY[mask])
trueY[mask].shape
#predY[mask].shape
mask.shape
predY.shape
predY

In [ ]:
trueX= valX[1,:]
trueX = np.expand_dims(trueX, axis=0)
trueX.shape
trueX = np.expand_dims(trueX, axis=0)
trueX.shape

In [ ]:
trueX = valX
#
trueX = np.expand_dims(trueX, axis=1)
trueX.shape

In [ ]:
trueY.shape

In [ ]:
predY[:]

In [ ]:
np.unique(valY)

In [ ]:
#predY = model.predict(trueX, verbose=0)
predY[:,5].shape
for i in range(0, 7):
    newpredY[:,i] = predY[:,i]

In [ ]:
predY

In [ ]:
model.output

In [ ]:
model.summary()

In [ ]:
import numpy as np
#from sklearn.metrics import classification
y_true = np.array([[1, 0, 1],
                    [0, 1, 0]])
y_pred = np.array([[1, 0, 0],
                    [0, 1, 1]])
multilabel_confusion_matrix(y_true, y_pred)

In [ ]:
patAcc

In [ ]:
castel = patAcc

In [ ]:
casette = patAcc

In [ ]:
random = patAcc

In [ ]:
randomvalpat = np.unique(pat_val)

In [ ]:
castelvalpat = np.unique(pat_val)

In [ ]:
casettevalpat = np.unique(pat_val)

In [ ]:
patAcc

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 2, 3, sharey=True)
f.suptitle('Per Patient Accuracy')


import pandas as pd
#patAccDir = "E:\SleepWell\ASSC\logs\lastAlldataChannel1_2019-01-19 18-37-49.652298/epoch22patAcc.csv"
#patAccDir = "E:/SleepWell/ASSC/logs/EEGcasetteonlyADGC_2019-01-21 14-52-02.119827/epoch8patAcc.csv"
#df = pd.read_csv(patAccDir,header=None)
from numpy import genfromtxt
my_data = patAcc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set(style='dark')
print(my_data)
your_data= my_data
print(your_data)
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

#objects = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19')
#objects = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19')
pat_valu = np.unique(pat_val)



data = pd.read_csv("patientnames.csv", header=None, delimiter=' ') 
a= data.iloc[:,0].values
name_val=[]
patient_list = np.unique(pat_valu)
for i in range (1,len(np.unique(pat_valu))+1):
    patHere = patient_list[i-1]
    name_val.append(a[ int(patHere) -1 ])
objects=tuple(name_val)  




f = plt.figure()

y_pos = np.arange(len(objects))
performance = your_data
 
sns.barplot(y_pos, performance,
#             align='center',
            alpha=1,
            color='teal'
           )




plt.xticks(y_pos, objects, rotation=60, fontsize=8)
plt.ylabel('Accuracy')
plt.title('Per Patient Accuracy')
plt.xlim(-.5,11.5)
plt.ylim(.2,.99)
plt.grid(linewidth=.2)
f.savefig("E:/SleepWell/fig/SleepCasettepat.pdf")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# plot a line, implicitly creating a subplot(111)
plt.plot([1,2,3])
# now create a subplot which represents the top plot of a grid
# with 2 rows and 1 column. Since this subplot will overlap the
# first, the plot (and its axes) previously created, will be removed
plt.subplot(121)
plt.show()

In [ ]:
(np.unique(randomvalpat))

#randomvalpat.type
fakevalpat= [24, 31, 10 ,16, 30 ,9 ,49 ,25 ,41 ,55 ,17 ,26 ,11 ,40 ,8 ,29 ,43 ,58 ,32]
np.asarray(fakevalpat)
fakevalpat=np.unique(fakevalpat)
print(fakevalpat)



name_val=[]
patient_list = fakevalpat

for i in range (1,len(np.unique(pat_valu1))+1):
    patHere = patient_list[i-1]
    name_val.append(a[ int(patHere) ])
objects=tuple(name_val)  

print(objects)

In [ ]:
#plt.subplot(131)


import pandas as pd
#patAccDir = "E:\SleepWell\ASSC\logs\lastAlldataChannel1_2019-01-19 18-37-49.652298/epoch22patAcc.csv"
#patAccDir = "E:/SleepWell/ASSC/logs/EEGcasetteonlyADGC_2019-01-21 14-52-02.119827/epoch8patAcc.csv"
#df = pd.read_csv(patAccDir,header=None)
from numpy import genfromtxt
my_data = patAcc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set(style='dark')
print(my_data)
your_data1= random
your_data2= casette
your_data3= castel
print(your_data)
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

#objects = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19')
#objects = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19')

pat_valu1 = fakevalpat
pat_valu2 = casettevalpat
pat_valu3 = castelvalpat



#pat_valu1 = np.unique(your_data1)
#pat_valu2 = np.unique(your_data2)
#pat_valu3 = np.unique(your_data3)


data = pd.read_csv("patientnames.csv", header=None, delimiter=' ') 
a = data.iloc[:,0].values

name_val1=[]
patient_list1 = pat_valu1

for i in range (1,len(np.unique(pat_valu1))+1):
    patHere = patient_list1[i-1]
    name_val1.append(a[ int(patHere) ])
objects1=tuple(name_val1)  

print(objects1)


name_val2=[]
patient_list2 = pat_valu2
for i in range (1,len(np.unique(pat_valu2))+1):
    patHere = patient_list2[i-1]-1
    name_val2.append(a[ int(patHere)  ])
objects2=tuple(name_val2)

print(objects2)

# name_val3=[]
# patient_list3 = np.unique(pat_valu3)
# for i in range (1,len(np.unique(pat_valu3))+1):
#     patHere = patient_list3[i-1]
#     name_val3.append(a[ int(patHere) -1 ]
#                     )
# objects3=tuple(name_val3)  
# print(objects3)


#f = plt.figure()

y_pos1 = np.arange(len(objects1))
y_pos2 = np.arange(len(objects2))
#y_pos3 = np.arange(len(objects3))


performance1 = your_data1
performance2 = your_data2
#performance3 = your_data3
    
    
# plt.figure(figsize=(12,12))

# plt.ylabel('Accuracy')
# plt.figure(figsize=(12,3))


f, axes = plt.subplots(1, 2, figsize=(8,4))    
sns.barplot(y_pos1, performance1, color='teal',  orient='v' , ax=axes[0])
axes[0].set_xticks(y_pos1)
axes[0].set_xticklabels(objects1,rotation=60,fontsize=9)
axes[0].set(ylim=[.4,.99],xlabel="(a)",ylabel="Per Patient Accuracy")
axes[0].grid(linewidth=.2)


# xticks(y_pos1, objects1, rotation=60, fontsize=8)

# plt.title('Per Patient Accuracy')
# 
# plt.xlim(-.5,11.5)
# plt.ylim(.4,.99)
 

sns.barplot(y_pos2, performance2, color='teal',  orient='v' , ax=axes[1])
plt.xticks(y_pos2, objects2, rotation=60, fontsize=8)
# plt.ylabel('Accuracy')
# plt.title('Per Patient Accuracy')
axes[1].set(ylim=[.4,.99],xlabel="(b)")
plt.xlim(-.5,11.5)
plt.ylim(.4,.99)
plt.figure(figsize=(3,8))
# plt.xlabel('(b)')
axes[1].grid(linewidth=.2)

f.tight_layout()

f.savefig("E:/SleepWell/fig/PerPatAccFinal4.pdf")
plt.show()






# sns.barplot(  y_pos3, performance3, color='teal',  orient='v' , ax=axes[2])    

# plt.xticks(y_pos, objects, rotation=60, fontsize=8)
# plt.ylabel('Accuracy')
# plt.title('Per Patient Accuracy')
# plt.xlim(-.5,11.5)
# plt.ylim(.2,.99)
    
# sns.barplot(y_pos, performance,
# #             align='center',
#             alpha=1,
#             color='teal'
#            )


# plt.xticks(y_pos, objects, rotation=60, fontsize=8)
# plt.ylabel('Accuracy')
# plt.title('Per Patient Accuracy')
# plt.xlim(-.5,11.5)
# plt.ylim(.2,.99)
# plt.grid(linewidth=.2)
# f.savefig("E:/SleepWell/fig/figuresPatAcc.pdf")




# plt.subplot(131, sharex=ax1, facecolor='red')
# plt.subplot(132, projection='polar')
# plt.subplot(133, projection='polar')

# add a red subplot that shares the x-axis with ax1
#plt.subplot(224, sharex=ax1, facecolor='red')

#delete ax2 from the figure
#plt.delaxes(ax2)

#add ax2 to the figure again
#plt.subplot(ax2)



In [ ]:
sns.barplot(y_pos, performance,
#             align='center',
            alpha=1,
            color='teal'
           )
plt.show()

In [ ]:
np.unique(pat_val)

In [ ]:
pat_val

In [ ]:
pat_valu1

In [ ]:
patient_list3

In [ ]:
y_pos2

In [ ]:
pat_val